In [1]:
# Notebook 2: generation of RNA-FM embeddings.
# We need:
# 1. Fasta file of all the unique sequences in the dataset (See Notebook 1).
# Connect to T4 Runtime with colab (Runtime -> change Runtime type -> T4)
# Always remember: Colab is not saving any data, you'll need to download the
# results at the end.

In [2]:
import pickle
import os
import numpy as np
import torch

In [3]:
!git clone https://github.com/ml4bio/RNA-FM.git

Cloning into 'RNA-FM'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 359 (delta 49), reused 44 (delta 43), pack-reused 293 (from 1)
Receiving objects: 100% (359/359), 16.07 MiB | 11.03 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [ ]:
# The fasta file of the unique sequences should be placed in a new directory
# in 'RNA-FM/redevelop/data/example'
# This is an example notebook, when you'll use your data, you need to change
# the diresctories in this notebook accordingly.

In [4]:
# After condacolab installation the colab kernel will be shutted down
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# In colab terminal, run: 'conda env create -f RNA-FM/environment.yml'
# It takes a while to install all the libraries needed for RNA-FM
# 5 to 10 minutes..

In [ ]:
# In colab terminal, run:
# 'source /usr/local/etc/profile.d/conda.sh && conda activate RNA-FM'

In [ ]:
# Now we can extract embeddings from sequences, in colab terminal run:

# MOVE IN THIS DIR: RNA-FM/redevelop/ then run:

'''
python launch/predict.py --config="pretrained/extract_embedding.yml" \
--data_path="./data/examples/example.fasta" --save_dir="./resuts/example1" \
--save_frequency 1 --save_embeddings
'''

In [ ]:
# Now we can extract embeddings from sequences, in colab terminal run:
'''
python launch/predict.py --config="pretrained/extract_embedding.yml" \
--data_path="./data/examples/unique_sequences_with_flipped_lncrna_pseudo.fasta" --save_dir="./resuts/lncrna_pseudo" \
--save_frequency 1 --save_embeddings
'''

In [1]:
!pip install torch
!pip install numpy
# After this two installations the kernerl should be restarted again

  Using cached torch-2.9.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2

In [6]:
# Time for pooling the embedding matrices.

# Load the pickle file
def load_pkl(file_path):
  with open(file_path, "rb") as f:
      data = pickle.load(f)
  return data


def read_fasta(fasta_path):
  """
  Read sequences and IDs from a FASTA file.

  Parameters:
      fasta_path (str): Path to the FASTA file.

  Returns:
      list of tuples: [(id, sequence), ...]
  """
  entries = []
  seq = []
  seq_id = None
  with open(fasta_path, "r") as f:
      for line in f:
          line = line.strip()
          if not line:
              continue
          if line.startswith(">"):
              if seq_id and seq:
                  entries.append((seq_id, "".join(seq)))
                  seq = []
              seq_id = line[1:].strip()  # remove ">"
          else:
              seq.append(line)
      if seq_id and seq:
          entries.append((seq_id, "".join(seq)))
  return entries


def get_sequence_embeddings(fasta_file, embeddings_paths):
  """
  Given a FASTA file of unique sequences,
  create a dictionary mapping each sequence to its embedding.

  The combined embedding is computed by:
    - Max pooling along the sequence length dimension (dim=0)
    - Average pooling along the sequence length dimension (dim=0)
    - Concatenating the two pooled results along the feature dimension (dim=-1)

  The embedding file for each sequence is expected to be named '{id}.npy'
  in the directory embeddings_paths.

  Parameters:
      fasta_file (str): Path to the FASTA file containing unique sequences.
      embeddings_paths (str): Path to the directory containing embedding files.

  Returns:
      dict: Dictionary where keys are sequences and values are the combined embeddings (torch.Tensor).

  Raises:
      ValueError: If duplicate sequences are found in the FASTA.
  """

  fasta_entries = read_fasta(fasta_file)
  #print(fasta_entries)
  sequence_embeddings = {}

  for seq_id, seq in fasta_entries:
      if seq in sequence_embeddings:
          raise ValueError(f"Duplicate sequence found in FASTA: {seq[:30]}...")
      #print(seq_id)

      # Build filename from ID
      embedding_file = os.path.join(embeddings_paths, f"{seq_id}.npy")
      if not os.path.exists(embedding_file):
          print(f"Warning: Embedding file not found for {seq_id}")
          continue

      embedding_np = np.load(embedding_file)
      embedding = torch.from_numpy(embedding_np)
      #print(len(embedding))

      # Pooling along sequence length
      max_pooled, _ = embedding.max(dim=0)
      avg_pooled = embedding.mean(dim=0)
      #print(len(max_pooled))
      #print(len(avg_pooled))

      combined_embedding = torch.cat([max_pooled, avg_pooled], dim=-1)
      #print(len(combined_embedding))
      #print(combined_embedding)

      # Store embedding (key = sequence, value = embedding)
      sequence_embeddings[seq] = combined_embedding
      #break

  return sequence_embeddings

In [8]:
import os
import numpy as np
import torch
import pickle
file_path_unique = 'RNA-FM/redevelop/data/examples/unique_sequences_with_flipped_lncrna_pseudo.fasta'
embeddings_paths = 'RNA-FM/redevelop/resuts/lncrna_pseudo/representations'
sequence_embeddings_dict = get_sequence_embeddings(file_path_unique, embeddings_paths)

out_path = 'lncrna-pseudo_dictionary_pooled_embeddings_RNAFM.p'
with open(out_path, 'wb') as f:
    pickle.dump(sequence_embeddings_dict, f)

In [9]:
sequence_embeddings_dict

{'CCGCUGAGCGAGGGCCCUCAUGAGAAGGGAAGGUCGCGUGGACAGGCUGCCAGGGGCAAGCGUUUUUAGAUUACUUCCUCCUUCUUGGCCCCAGAUGCACUUGGUUGCACGCUCUGGUUCCCAAACCCCCGGCCGCUGGCUUAUGCAAAUCACUUAGGUACAUGCAAAAGUAUCCCUUCUCCCGGAGCGCCAUUGGCCCGGGGAGGUCUCGAGCUCAUUACUAUGCAGAGAGGAGAGCCGCCAUUGGCCAAGAGGAGGACCAGAGGGGCGUGUUUCUCGGGCAAAUUGGAUCUCCUAAAUUGGAUGACCUGGCUAAUGUUAAAACAUACUUCGUGAUCUGUAGAACGUAAGAGAAAAUUUAGAAAAUGUUUGAUAUGAAAGAGAAAAUGAAGGAUGUUGCGAUAAUAGGGAAAAACUUUAUAGUAAUUCUACCAAUUUGCCCCUUUUAUUCUAAAAUAACACAAACUUGUCAAGCUUUUCAACAUUUUUUGAAUUUUUUUUUUUUUUUUUUGAGAUGGAGUUUCACUCUGUUGCCCA': tensor([ 0.6807,  0.5657,  0.4613,  ..., -0.2706,  0.0298,  0.0673]),
 'AACACUUUGUCUUCAGUGUCAAGACAAAUAAAACGUUACUACAUUUGAAGUGUUAAAAAGGAAUGUCUAUUUUUUAUGUUUGGAAAGUGACGAUAUAGGUUCUUUAGACAUUUGAAGUUUGUAACAUCUGGACGGACAGAUUGCAUUUACCUUGCGUCUGUUACAAUCACCACAGGAGUACGCUCAAUCGGGUCGGACCCCACCACCGAGGGAAGGGCGUAAACACUCCUCUACCGUCGCUCCUACGACUUGGACACUUCCCCCACGGACCUCCCUGCUCAGAACGUUGAGUCACACCACGUGGACGGUCGUCGUAGGUAGUGGAUCCUCGAACAACCUUUGUUCCUCGGAGUCGGGGGUCGGAGACCUGCGACGACGACGAGGUG